# 2D Planet Simulator to Generate Data for ML Inferred Physics

Author: Craig Boger
06/01/2020

Looks like the simulator is 2D, but can be changed to 3D if needed.

This is a script to generate some quick simulated data for orbiting objects in a 2D space to fead into a neural network to predict the next position of a body traveling through a system.

## Straight Up Just Stealing Someone's Code and Trying to Run It

Credit to benrules2: https://gist.github.com/benrules2/220d56ea6fe9a85a4d762128b11adfba

In [1]:
import math
import random
%matplotlib widget
import matplotlib.pyplot as plot
from mpl_toolkits.mplot3d import Axes3D

class point:
    def __init__(self, x,y,z):
        self.x = x
        self.y = y
        self.z = z

class body:
    def __init__(self, location, mass, velocity, name = ""):
        self.location = location
        self.mass = mass
        self.velocity = velocity
        self.name = name

def calculate_single_body_acceleration(bodies, body_index):
    G_const = 6.67408e-11 #m3 kg-1 s-2
    acceleration = point(0,0,0)
    target_body = bodies[body_index]
    for index, external_body in enumerate(bodies):
        if index != body_index:
            r = (target_body.location.x - external_body.location.x)**2 + (target_body.location.y - external_body.location.y)**2 + (target_body.location.z - external_body.location.z)**2
            r = math.sqrt(r)
            tmp = G_const * external_body.mass / r**3
            acceleration.x += tmp * (external_body.location.x - target_body.location.x)
            acceleration.y += tmp * (external_body.location.y - target_body.location.y)
            acceleration.z += tmp * (external_body.location.z - target_body.location.z)

    return acceleration

def compute_velocity(bodies, time_step = 1):
    for body_index, target_body in enumerate(bodies):
        acceleration = calculate_single_body_acceleration(bodies, body_index)

        target_body.velocity.x += acceleration.x * time_step
        target_body.velocity.y += acceleration.y * time_step
        target_body.velocity.z += acceleration.z * time_step 


def update_location(bodies, time_step = 1):
    for target_body in bodies:
        target_body.location.x += target_body.velocity.x * time_step
        target_body.location.y += target_body.velocity.y * time_step
        target_body.location.z += target_body.velocity.z * time_step

def compute_gravity_step(bodies, time_step = 1):
    compute_velocity(bodies, time_step = time_step)
    update_location(bodies, time_step = time_step)

def plot_output(bodies, outfile = None):
    fig = plot.figure()
    colours = ['r','b','g','y','m','c']
    ax = fig.add_subplot(1,1,1, projection='3d')
    max_range = 0
    for current_body in bodies: 
        max_dim = max(max(current_body["x"]),max(current_body["y"]),max(current_body["z"]))
        if max_dim > max_range:
            max_range = max_dim
        ax.plot(current_body["x"], current_body["y"], current_body["z"], c = random.choice(colours), label = current_body["name"])        
    
    ax.set_xlim([-max_range,max_range])    
    ax.set_ylim([-max_range,max_range])
    ax.set_zlim([-max_range,max_range])
    ax.legend()        

    if outfile:
        plot.savefig(outfile)
    else:
        plot.show()

def run_simulation(bodies, names = None, time_step = 1, number_of_steps = 10000, report_freq = 100):

    #create output container for each body
    body_locations_hist = []
    for current_body in bodies:
        body_locations_hist.append({"x":[], "y":[], "z":[], "name":current_body.name})
        
    for i in range(1,number_of_steps):
        compute_gravity_step(bodies, time_step = 1000)            
        
        if i % report_freq == 0:
            for index, body_location in enumerate(body_locations_hist):
                body_location["x"].append(bodies[index].location.x)
                body_location["y"].append(bodies[index].location.y)           
                body_location["z"].append(bodies[index].location.z)       

    return body_locations_hist        
            
#planet data (location (m), mass (kg), velocity (m/s)
sun = {"location":point(0,0,0), "mass":2e30, "velocity":point(0,0,0)}
mercury = {"location":point(0,5.7e10,0), "mass":3.285e23, "velocity":point(47000,0,0)}
venus = {"location":point(0,1.1e11,0), "mass":4.8e24, "velocity":point(35000,0,0)}
earth = {"location":point(0,1.5e11,0), "mass":6e24, "velocity":point(30000,0,0)}
mars = {"location":point(0,2.2e11,0), "mass":2.4e24, "velocity":point(24000,0,0)}
jupiter = {"location":point(0,7.7e11,0), "mass":1e28, "velocity":point(13000,0,0)}
saturn = {"location":point(0,1.4e12,0), "mass":5.7e26, "velocity":point(9000,0,0)}
uranus = {"location":point(0,2.8e12,0), "mass":8.7e25, "velocity":point(6835,0,0)}
neptune = {"location":point(0,4.5e12,0), "mass":1e26, "velocity":point(5477,0,0)}
pluto = {"location":point(0,3.7e12,0), "mass":1.3e22, "velocity":point(4748,0,0)}
# TODO: Add random sattellite here.
satellite_1 = {"location":point(1e5,3.7e5,0), "mass":1.7e1, "velocity":point(4748,0,0)}

if __name__ == "__main__":

    #build list of planets in the simulation, or create your own
    bodies = [
        body( location = sun["location"], mass = sun["mass"], velocity = sun["velocity"], name = "sun"),
        body( location = earth["location"], mass = earth["mass"], velocity = earth["velocity"], name = "earth"),
        body( location = mars["location"], mass = mars["mass"], velocity = mars["velocity"], name = "mars"),
        body( location = venus["location"], mass = venus["mass"], velocity = venus["velocity"], name = "venus"),
        body( location = mercury["location"], mass = mercury["mass"], velocity = mercury["velocity"], name = "mercury"),
        body( location = jupiter["location"], mass = jupiter["mass"], velocity = jupiter["velocity"], name = "jupiter"),
        
        #body( location = satellite_1["location"], mass = satellite_1["mass"], velocity = satellite_1["velocity"], name = "sattellite_1")
        ]
    
    # Original defaults of simulation
    # motions = run_simulation(bodies, time_step = 100, number_of_steps = 80000, report_freq = 1000)
    # Try messing with report frequency to get more data.
    motions = run_simulation(bodies, time_step = 100, number_of_steps = 300000, report_freq = 100)
    plot_output(motions, outfile = 'orbits.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Take motions data from the above simulation and convert it to a Pandas dataframe.  The "motions" output is a list of python dictionaries that can be converted into a dataframe and then manipulated.

In [2]:
import pandas as pd
import numpy as np

motions_df = pd.DataFrame(motions)
motions_df.head(100)

,x,y,z,name
0,"[6.15145359812474, 49.209722543602815, 166.052...","[5958.957631921161, 23717.444806285053, 53274....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",sun
1,"[2999802268.825811, 5998418130.70297, 89946616...","[149970049677.5866, 149880803782.63992, 149732...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",earth
2,"[2399949856.0622263, 4799598822.151186, 719864...","[219986083392.02606, 219944610590.4192, 219875...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",mars
3,"[3499415066.5380883, 6995320910.72392, 1048421...","[109944304322.01828, 109778376408.54349, 10950...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",venus
4,"[4694355206.133197, 9354859806.40224, 13947776...","[56792631228.91187, 56175842844.06461, 5515329...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",mercury
5,"[1299999366.5820706, 2599994932.2779655, 38999...","[769998863064.6425, 769995474775.1132, 7699898...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",jupiter


In [3]:
# Trying to separate out each row of list or dataframe into its own dataframe.
# Will later put these dataframes back together into 1 large dataframe.

motions_df_list = []
for body in motions:
    motions_df_list.append(pd.DataFrame(body))

In [4]:
motions_df_list[3]

,x,y,z,name
0,3.499415e+09,1.099443e+11,0.0,venus
1,6.995321e+09,1.097784e+11,0.0,venus
2,1.048421e+10,1.095024e+11,0.0,venus
3,1.396259e+10,1.091166e+11,0.0,venus
4,1.742697e+10,1.086215e+11,0.0,venus
...,...,...,...,...
2994,-5.346618e+10,8.152006e+10,0.0,venus
2995,-5.087488e+10,8.385649e+10,0.0,venus
2996,-4.821074e+10,8.611085e+10,0.0,venus
2997,-4.547635e+10,8.828085e+10,0.0,venus


In [5]:
# Combine the dataframes into a single, large dataframe.
# Can later choose a planet to be the target we train to predict.
complete_motion_df = None

for body in motions_df_list:
    # Append name of body to each column and remove the name column
    body_name = body.loc[0, "name"]
    body.columns = [body_name + "_x",
                    body_name + "_y",
                    body_name + "_z",
                    "name"]
    # Add current body to the complete dataframe.
    complete_motion_df = pd.concat([complete_motion_df, body.iloc[:, 0:3]], axis=1)

complete_motion_df.head(100)

,sun_x,sun_y,sun_z,earth_x,earth_y,earth_z,mars_x,mars_y,mars_z,venus_x,venus_y,venus_z,mercury_x,mercury_y,mercury_z,jupiter_x,jupiter_y,jupiter_z
0,6.151454e+00,5.958958e+03,0.0,2.999802e+09,1.499700e+11,0.0,2.399950e+09,2.199861e+11,0.0,3.499415e+09,1.099443e+11,0.0,4.694355e+09,5.679263e+10,0.0,1.299999e+09,7.699989e+11,0.0
1,4.920972e+01,2.371744e+04,0.0,5.998418e+09,1.498808e+11,0.0,4.799599e+09,2.199446e+11,0.0,6.995321e+09,1.097784e+11,0.0,9.354860e+09,5.617584e+10,0.0,2.599995e+09,7.699955e+11,0.0
2,1.660524e+02,5.327429e+04,0.0,8.994662e+09,1.497323e+11,0.0,7.198646e+09,2.198756e+11,0.0,1.048421e+10,1.095024e+11,0.0,1.394778e+10,5.515330e+10,0.0,3.899983e+09,7.699898e+11,0.0
3,3.934971e+02,9.462756e+04,0.0,1.198735e+10,1.495246e+11,0.0,9.596791e+09,2.197790e+11,0.0,1.396259e+10,1.091166e+11,0.0,1.843961e+10,5.373112e+10,0.0,5.199959e+09,7.699819e+11,0.0
4,7.682695e+02,1.477745e+05,0.0,1.497529e+10,1.492578e+11,0.0,1.199373e+10,2.196549e+11,0.0,1.742697e+10,1.086215e+11,0.0,2.279721e+10,5.191794e+10,0.0,6.499921e+09,7.699718e+11,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4.120216e+06,5.305332e+07,0.0,1.441445e+11,-4.894566e+10,0.0,1.875072e+11,1.033952e+11,0.0,1.578896e+10,-1.109147e+11,0.0,3.050287e+10,-4.133121e+10,0.0,1.242401e+11,7.596460e+11,0.0
96,4.240064e+06,5.414771e+07,0.0,1.431961e+11,-5.174369e+10,0.0,1.885863e+11,1.011757e+11,0.0,1.238223e+10,-1.113496e+11,0.0,2.606634e+10,-4.407945e+10,0.0,1.255224e+11,7.594296e+11,0.0
97,4.362014e+06,5.525284e+07,0.0,1.421936e+11,-5.452212e+10,0.0,1.896398e+11,9.894245e+10,0.0,8.963777e+09,-1.116789e+11,0.0,2.137174e+10,-4.639053e+10,0.0,1.268044e+11,7.592110e+11,0.0
98,4.486067e+06,5.636871e+07,0.0,1.411373e+11,-5.727992e+10,0.0,1.906674e+11,9.669569e+10,0.0,5.536832e+09,-1.119023e+11,0.0,1.646398e+10,-4.823808e+10,0.0,1.280860e+11,7.589902e+11,0.0


In [6]:
complete_motion_df.shape

(2999, 18)

At this point, we have a single dataframe with all bodies and all positions with each time step as the index of our rows.

# Try a Quick Nerual Net for Predicting Jupiter's Position

### Imports

In [7]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
# Probably not needed since not using regressor or doing any feature engineering.
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Import Keras
from tensorflow import keras

# to make this notebook's output stable across runs
np.random.seed(42)

# Common imports
import numpy as np
import os

In [8]:
tf.__version__

'2.2.0'

In [9]:
keras.__version__

'2.3.0-tf'

### Convert Pandas Dataframe to Tensorflow Dataset

https://www.tensorflow.org/tutorials/load_data/pandas_dataframe

In [10]:
# Assuming last 3 columns in the dataframe are the target x,y, and z values.  
target = complete_motion_df.iloc[:,-3:]
# Drop target from main dataframe.
complete_motion_df.drop(complete_motion_df.iloc[:,-3:], axis = 1, inplace = True)
target.head(5)

,jupiter_x,jupiter_y,jupiter_z
0,1.299999e+09,7.699989e+11,0.0
1,2.599995e+09,7.699955e+11,0.0
2,3.899983e+09,7.699898e+11,0.0
3,5.199959e+09,7.699819e+11,0.0
4,6.499921e+09,7.699718e+11,0.0


NEED TO SCALE DATA HERE

In [11]:
# Convert training and target data into tensorflow dataset.
full_dataset = tf.data.Dataset.from_tensor_slices((complete_motion_df.values, target.values))
# Shuffle the fulle dataset.  Set the buffer to just be the size of the dataset for now.
# Might need to reduce buffer later on with larger datasets.
full_dataset = full_dataset.shuffle(len(complete_motion_df))

In [12]:
# Setup train, validation, and test splits
DATASET_SIZE = len(complete_motion_df)
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

In [13]:
# Select the specified number of rows for training, validation, and test
# https://stackoverflow.com/questions/51125266/how-do-i-split-tensorflow-datasets#:~:text=Take%3A,count%20elements%20from%20this%20dataset.
# Take a number of elements to create the full training set
train_dataset = full_dataset.take(train_size)
# Create a set that has both the validation and testing data in it by skipping over training data
test_dataset = full_dataset.skip(train_size)
# Split the above data into validation and testing data.
val_dataset = test_dataset.skip(test_size)
test_dataset = test_dataset.take(test_size)

In [15]:
# Print out training dataset.
for feat, targ in full_dataset.take(5):
    print('Features: {}, Target: {}'.format(feat, targ))

Features: [ 1.87252388e+10  5.31172243e+09  0.00000000e+00  1.29571843e+11
 -9.99696052e+10  0.00000000e+00 -4.39686524e+10 -1.83817950e+11
  0.00000000e+00 -9.18745716e+10  1.44456965e+10  0.00000000e+00
 -3.49048121e+10  1.35142787e+10  0.00000000e+00], Target: [-6.56699486e+11 -2.92050294e+11  0.00000000e+00]
Features: [ 3.04448130e+09  4.59849760e+09  0.00000000e+00  1.45511155e+11
  5.49990496e+10  0.00000000e+00 -3.44960950e+10  2.21132966e+11
  0.00000000e+00  6.76720265e+10  9.38479536e+10  0.00000000e+00
  2.15821688e+10  5.82923110e+10  0.00000000e+00], Target: [ 7.29111385e+11 -1.49635082e+11  0.00000000e+00]
Features: [ 1.78053570e+10  5.85442295e+09  0.00000000e+00  9.05512317e+10
  1.37232267e+11  0.00000000e+00  1.77718794e+11 -1.18450590e+11
  0.00000000e+00  1.22157604e+11 -3.31016491e+10  0.00000000e+00
  5.33155504e+10 -3.16251591e+10  0.00000000e+00], Target: [-5.94150115e+11 -4.00724101e+11  0.00000000e+00]
Features: [ 4.04148730e+09  5.32139669e+09  0.00000000e+00

In [38]:
# Function to generate model (neural network)
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(15, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(3, activation='linear')
  ])

  model.compile(optimizer='sgd',
                loss='mean_squared_error',
                metrics=['accuracy'])
  return model

In [39]:
# Compile model
model = get_compiled_model()

In [40]:
# Train model
# history = model.fit(train_dataset, epochs=15, validation_data = val_dataset)
history = model.fit(train_dataset, epochs=15)

Epoch 1/15

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



ValueError: in user code:

    C:\tools\Anaconda3\envs\py37_ML_Playground\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\tools\Anaconda3\envs\py37_ML_Playground\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\tools\Anaconda3\envs\py37_ML_Playground\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\tools\Anaconda3\envs\py37_ML_Playground\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\tools\Anaconda3\envs\py37_ML_Playground\lib\site-packages\tensorflow\python\keras\engine\training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_losses=self.losses)
    C:\tools\Anaco

In [ ]:
# Evaluate the model
mse_test = model.evaluate(test_dataset)